# Easom function

---
Description:

- Optimization (min)
- Single-objective
- Constraints (no)
---

The general equation is given by:

- $f(x, y) = -\cos(x)\cos(y)\exp\{ -((x-\pi)^2 + (y-\pi)^2) \}$,

with  $-100 \le x, y \le +100$, and global minimum found at: $f(\pi, \pi) = -1.0$.

### First we import python libraries and set up the directory of our code.

In [1]:
import os, sys
import numpy as np
from math import fsum, isclose

PROJECT_DIR = os.path.abspath('..')
sys.path.append(PROJECT_DIR)

### Here we import all our custom GA code.

In [2]:
# Import main classes.
from pygenalgo.genome.gene import Gene
from pygenalgo.genome.chromosome import Chromosome
from pygenalgo.engines.island_model_ga import IslandModelGA

# Import Selection Operator(s).
from pygenalgo.operators.selection.linear_rank_selector import LinearRankSelector

# Import Crossover Operator(s).
from pygenalgo.operators.crossover.uniform_crossover import UniformCrossover

# Import Mutation Operator(s).
from pygenalgo.operators.mutation.gaussian_mutator import GaussianMutator

# Import Migration Operator(s).
from pygenalgo.operators.migration.clockwise_migration import ClockwiseMigration

### Define the Easom function, which plays also the role of the 'fitness' function.

In addition, we define the 'boundary_xy' which takes the role of the 'random()' method of the Genes. Every time
we want to 'mutate' a gene this function will be called and assign a 'valid', but random value for the gene.

In [3]:
# Easom function.
def fun_Easom(chromosome, f_min: bool = True):
    
    # Extract the data values as 'x' and 'y', for parsimony.
    x, y = [gene.value for gene in chromosome.genome]
    
    # Calculate the function value.
    f_val = -np.cos(x) * np.cos(y) * np.exp(-((x - np.pi)**2 + (y - np.pi)**2))

    # Condition for termination.
    solution_found = isclose(f_val, -1.0, rel_tol=1.0e-5)

    # Assign the fitness value (check for minimization).
    fit_value = -f_val if f_min else f_val
    
    # Return the solution tuple.
    return fit_value, solution_found
# _end_def_

# Random boundary function: ~U(-100, +100).
boundary_xy = lambda: np.random.uniform(-100.0, 100.0)

Here we set the GA parameters, such as number of genes, number of chromosomes, etc.

In [4]:
# Define the number of chromosomes.
N = 200

# Draw random samples for the initial points.
xy_init = np.random.uniform(-25.0, +25.0, size=(N, 2))

# Initial population.
population = [Chromosome([Gene(xy_init[i, 0], boundary_xy),
                          Gene(xy_init[i, 1], boundary_xy)], np.nan, True)
              for i in range(N)]

# Create the IslandModelGA object that will carry on the optimization.
test_GA = IslandModelGA(initial_pop=population,
                        fit_func=fun_Easom, num_islands=4,
                        select_op=LinearRankSelector(),
                        mutate_op=GaussianMutator(),
                        crossx_op=UniformCrossover(),
                        migrate_op=ClockwiseMigration())

### Optimization process.

Here we call the GA object (either directly, or through the method run()). We set a number of parameter, such as the maximum iterations (i.e. epochs), tolerance for the fitness convergences, etc.

In [5]:
test_GA(epochs=1500, elitism=True, f_tol=1.0e-6, allow_migration=True, adapt_probs=True)

Parallel evolution in progress with 4 islands ...
Final Avg. Fitness = 0.2278, Spread = 0.4155.
Elapsed time: 2.178 seconds.


In [6]:
# Extract the optimal solution from the GA.
optimal_solution = test_GA.best_chromosome()

# Extract the fitness value from the optimal solution.
optimal_fit, _ = fun_Easom(optimal_solution, f_min=False)

# Display the (final) optimal value.
print(f"Minimum Found: {optimal_fit:.5f}\n")

# Display each gene value separately.
for i, xi in enumerate(optimal_solution.genome):
    print(f"x{i} = {xi.value:.5f}")
# _end_for_

# True minimum: f(\pi, \pi) = -1.0

Minimum Found: -1.00000

x0 = 3.14149
x1 = 3.14061


### End of file